In [1]:
import os
from imageio import imread
from pathlib import Path
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

(train_images_org, train_labels), (test_images_org, test_labels) = datasets.cifar10.load_data()
# train_images_img, test_images_img = train_images_org.astype(dtype=np.complex64) / 255.0, test_images_org.astype(dtype=np.complex64) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

2023-08-06 01:17:56.042533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 01:17:57.208626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
def dataset_split(train_img, train_labels):

    train_dat, train_label, val_dat, val_labels = train_test_split(train_img, train_labels, test_size=.1) 

    return train_dat, train_label, val_dat, val_labels  

In [5]:
train_images_org, val_images_org, train_labels, val_labels = dataset_split(train_images_org, train_labels)

In [6]:
train_labels.shape

(45000, 10)

In [7]:
from tensorflow.keras import models
import tensorflow.keras.layers as layers

def get_tf_sequential_model():
    model = models.Sequential()
    model.add(layers.Input(shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(10, activation='softmax'))
    return model

get_tf_sequential_model().summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

2023-08-06 01:17:59.424518: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-06 01:17:59.741739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-06 01:17:59.742021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
import cvnn.layers as complex_layers

def get_cvnn_sequential_model():
    model = models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(32, 32, 3)))   # Always use ComplexInput at the start
    model.add(complex_layers.ComplexConv2D(32, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexConv2D(64, (3, 3), activation="cart_relu"))
    model.add(complex_layers.ComplexFlatten())
    model.add(complex_layers.ComplexDense(64, activation="cart_relu"))
    model.add(complex_layers.ComplexDense(10, activation='softmax_real_with_abs'))
    return model

get_cvnn_sequential_model().summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 complex_conv2d (ComplexCon  (None, 30, 30, 32)        1792      
 v2D)                                                            
                                                                 
 complex_max_pooling2d (Com  (None, 15, 15, 32)        0         
 plexMaxPooling2D)                                               
                                                                 
 complex_conv2d_1 (ComplexC  (None, 13, 13, 64)        36992     
 onv2D)                                                          
                                                                 
 complex_max_pooling2d_1 (C  (None, 6, 6, 64)          0         
 omplexMaxPooling2D)                                             
                                                                 
 complex_conv2d_2 (ComplexC  (None, 4, 4, 64)         

#### Adding a imaginary component into the real data and running the value and using CVNN

In [9]:
# #Running the CVNN model on the data

# from tensorflow import losses, metrics

# model_cvnn = get_cvnn_sequential_model()
# model_cvnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    
# history = model_cvnn.fit(train_images_img, train_labels, epochs=100, validation_data=(test_images_img, test_labels),batch_size=32)

In [10]:
#Running an ANN on the data while ignoring the imaginary values

model_ann = get_tf_sequential_model()
model_ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_ann.fit(train_images_org, train_labels, epochs=10, validation_data=(val_images_org, val_labels),batch_size=32)

Epoch 1/10


2023-08-06 01:18:02.563464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-06 01:18:03.645414: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x879d420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-06 01:18:03.645450: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-08-06 01:18:03.651433: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-06 01:18:03.796529: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1407/1407 [==============================] - 10s 4ms/step - loss: 1.8901 - accuracy: 0.3484 - val_loss: 1.5228 - val_accuracy: 0.4512
Epoch 2/10
1407/1407 [==============================] - 6s 4ms/step - loss: 1.4077 - accuracy: 0.4936 - val_loss: 1.4040 - val_accuracy: 0.5124
Epoch 3/10
1407/1407 [==============================] - 5s 4ms/step - loss: 1.2527 - accuracy: 0.5570 - val_loss: 1.2303 - val_accuracy: 0.5644
Epoch 4/10
1407/1407 [==============================] - 5s 4ms/step - loss: 1.1556 - accuracy: 0.5932 - val_loss: 1.1869 - val_accuracy: 0.5844
Epoch 5/10
1407/1407 [==============================] - 5s 4ms/step - loss: 1.0632 - accuracy: 0.6269 - val_loss: 1.1242 - val_accuracy: 0.6120
Epoch 6/10
1407/1407 [==============================] - 6s 4ms/step - loss: 0.9997 - accuracy: 0.6507 - val_loss: 1.1107 - val_accuracy: 0.6220
Epoch 7/10
 231/1407 [===>..........................] - ETA: 4s - loss: 0.9230 - accuracy: 0.6780

In [ ]:
loss, accu = model_ann.evaluate(test_images_org, test_labels)

313/313 [==============================] - 1s 4ms/step - loss: 1.0624 - accuracy: 0.6578


In [ ]:
print(f"The loss in the ANN is:{loss}, the accuracy is: {accu}")

The loss in the ANN is:1.0623869895935059 0.657800018787384


#### Now mapping the data from the real plane to the complex plane using Hilberts transform

In [ ]:
import scipy

train_images_hil, val_images_hil, test_images_hil = scipy.signal.hilbert(train_images_org / 255), \
     scipy.signal.hilbert(val_images_org / 255), scipy.signal.hilbert(test_images_org / 255)

In [ ]:
#Running the CVNN model on the hilbert data

from tensorflow import losses, metrics

model_cvnn = get_cvnn_sequential_model()
model_cvnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')
    
history = model_cvnn.fit(train_images_hil, train_labels, epochs=10, validation_data=(val_images_hil, val_labels),batch_size=32)

2023-08-06 01:08:40.450017: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2211840000 exceeds 10% of free system memory.
2023-08-06 01:08:42.269615: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2211840000 exceeds 10% of free system memory.


Epoch 1/10
1407/1407 [==============================] - 27s 17ms/step - loss: 1.9091 - accuracy: 0.3044 - val_loss: 1.6510 - val_accuracy: 0.4078
Epoch 2/10
1407/1407 [==============================] - 24s 17ms/step - loss: 1.5382 - accuracy: 0.4456 - val_loss: 1.7767 - val_accuracy: 0.3778
Epoch 3/10
1407/1407 [==============================] - 23s 17ms/step - loss: 1.3625 - accuracy: 0.5134 - val_loss: 1.3889 - val_accuracy: 0.4998
Epoch 4/10
1407/1407 [==============================] - 25s 18ms/step - loss: 1.2482 - accuracy: 0.5586 - val_loss: 1.6600 - val_accuracy: 0.4124
Epoch 5/10
1407/1407 [==============================] - 25s 17ms/step - loss: 1.1599 - accuracy: 0.5909 - val_loss: 1.2432 - val_accuracy: 0.5462
Epoch 6/10
1407/1407 [==============================] - 24s 17ms/step - loss: 1.0810 - accuracy: 0.6201 - val_loss: 1.3209 - val_accuracy: 0.5476
Epoch 7/10
1407/1407 [==============================] - 24s 17ms/step - loss: 1.0187 - accuracy: 0.6434 - val_loss: 1.4338 -

In [ ]:
loss, accu = model_cvnn.evaluate(test_images_hil, test_labels)

2023-08-06 01:15:33.110822: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.
2023-08-06 01:15:43.369305: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 468.75MiB (rounded to 491520000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-06 01:15:43.369420: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-06 01:15:43.369458: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 98, Chunks in use: 98. 24.5KiB allocated for chunks. 24.5KiB in use in bin. 8.5KiB client-requested in use in bin.
2023-08-06 01:15:43.369488: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 1, Chunks in use: 0. 512B

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

amework/bfc_allocator.cc:1095] InUse at 7f7f481af100 of size 2560 next 97
2023-08-06 01:15:43.372164: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481afb00 of size 2560 next 98
2023-08-06 01:15:43.372185: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0500 of size 256 next 109
2023-08-06 01:15:43.372208: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0600 of size 256 next 110
2023-08-06 01:15:43.372239: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0700 of size 256 next 111
2023-08-06 01:15:43.372261: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0800 of size 256 next 114
2023-08-06 01:15:43.372282: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0900 of size 256 next 118
2023-08-06 01:15:43.372303: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7f7f481b0a00 of size 256 next 119
2023-08-06 01:15:43.372324: I tensorflow/tsl/framework/bfc_allocator.cc:1095] I

In [ ]:
print(f"The loss in the CVNN is:{loss}, the accuracy is: {accu}")